In [ ]:
#!/usr/bin/env python3

from ipywidgets import VBox, HBox, Button, Output, Layout
from IPython.display import display, HTML
import base64

# --- Define projects and images ---
projects = {
    "TOXICITY PREDICITION": {"desc": "Predict toxicity for molecules based on their chemical properties using a Feed Forward Neural Network", "image": "project_image1.png"},
    "PUBLIC WATER USAGE": {"desc": "Description for project 2.", "image": "project_image2.png"},
    "SUSTAINABLE HOUSING": {"desc": "Description for project 3.", "image": "project_image3.png"},
    "INCOME AND EXPENSE": {"desc": "Description for project 4.", "image": "project_image4.png"}
}

# --- Thumbnail settings ---
thumbnail_width = "200px"
thumbnail_height = "150px"

# --- Helper: convert image to base64 ---
def image_to_base64(path):
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

# --- Output area for project details ---
details = Output()

# --- Create thumbnail buttons ---
buttons = []

for name, info in projects.items():
    img_b64 = image_to_base64(info["image"])
    image_uri = f"data:image/png;base64,{img_b64}"

    # Create button
    b = Button(
        description="",
        layout=Layout(width=thumbnail_width, height=thumbnail_height, margin="16px"),
        style=dict(button_color="transparent"),
        tooltip=name
    )

    # Add CSS class for image background and overlay
    css_class = f"thumb-{name.replace(' ', '_')}"
    custom_css = f"""
    <style>
    .{css_class} {{
        background-image: url('{image_uri}');
        background-size: cover;
        background-position: center;
        position: relative;
        font-weight: bold;
        color: white;
        overflow: hidden;
    }}
    .{css_class}::after {{
        content: '{name}';
        display: flex;
        align-items: center;
        justify-content: center;
        width: 100%;
        height: 60px;
        background-color: rgba(0,0,128,0.3);
        position: absolute;
        bottom: 0;
        left: 0;
        font-size: 20px;
        font-family: Arial Black, sans-serif;
        color: white;                     
        -webkit-text-stroke: 0.5px grey;  
        text-align: center;
        white-space: normal;
        line-height: 1.2em;
    }}
    .{css_class}:hover {{
        transform: scale(1.05);
        box-shadow: 0 0 10px #888;
        cursor: pointer;
        transition: transform 0.2s, box-shadow 0.2s;
    }}
    </style>
    """
    display(HTML(custom_css))
    b.add_class(css_class)
    buttons.append(b)

    # Click behavior with blue banner behind the title
    def on_click(b, name=name):
        details.clear_output()
        with details:
            info = projects[name]
            display(HTML(f"""
            <div style="background-color:#000080; color:white; font-family: Arial Black, sans-serif; font-size: 12px; padding:20px 20px; border-radius:10px;">
                <h3 style="margin:0;">{name}</h3>
            </div>
            <p style='color:#5A5A5A; padding:20px 20px'>{info['desc']}</p>
            <hr>
            <p><em>More details coming soon...</em></p>
            """))
    b.on_click(on_click)

# --- Layout thumbnails in 2x2 grid ---
grid = VBox([
    HBox(buttons[:2], layout=Layout(justify_content="center")),
    HBox(buttons[2:], layout=Layout(justify_content="center"))
], layout=Layout(width="35%", padding="10px"))

# --- Right column for project details ---
right_column = VBox([details], layout=Layout(width="65%", border="0px solid gray", padding="30px"))

# --- Combine into main UI ---
ui = HBox([grid, right_column], layout=Layout(width="100%", height="500px"))
display(ui)
